In [48]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import requests
import re
from fastai.text import *
import torch

In [3]:
book = requests.get('https://www.gutenberg.org/files/36/36-0.txt').text

In [4]:
start_iter = re.finditer('BOOK ONE', book)

In [5]:
next(start_iter)

<_sre.SRE_Match object; span=(1067, 1075), match='BOOK ONE'>

Commenting out some print lines here to shrink the notebook -- I did some exploration to figure out where the actual story began and ended (excluding table of contents, acknowledgements, etc.)

In [6]:
# book[1067:]

In [7]:
next(start_iter)

<_sre.SRE_Match object; span=(1920, 1928), match='BOOK ONE'>

In [8]:
# book[1920:]

In [9]:
book = book[1920:]

In [10]:
end_iter = re.finditer("among the dead.", book)

In [11]:
next(end_iter)

<_sre.SRE_Match object; span=(64109, 64124), match='among the dead '>

In [12]:
# book[:64125]

In [13]:
next(end_iter)

<_sre.SRE_Match object; span=(345594, 345609), match='among the dead.'>

In [14]:
# book[:345609]

In [15]:
book = book[:345609]

In [16]:
book = re.sub(r'[\r\n]+', ' ', str(book))

In [17]:
# book

In [18]:
tokens = [i.replace(',', '') for i in book.split('.')]

In [19]:
# tokens

In [20]:
tokens[:10]

['BOOK ONE THE COMING OF THE MARTIANS I',
 ' THE EVE OF THE WAR',
 ' No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâ\x80\x99s and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water',
 ' With infinite complacency men went to and fro over this globe about their little affairs serene in their assurance of their empire over matter',
 ' It is possible that the infusoria under the microscope do the same',
 ' No one gave a thought to the older worlds of space as sources of human danger or thought of them only to dismiss the idea of life upon them as impossible or improbable',
 ' It is curious to recall some of the mental habits of those departed days',
 ' At most terrestrial men fa

In [21]:
! rm book_text.csv

In [22]:
with open('book_text.csv', 'a+') as file:
    for sentence in tokens:
        file.write(sentence + '\n')

In [23]:
with open('book_text.csv') as file:
    lines = file.readlines()[:5]
print(lines[0])
print(lines[1])
print(lines[2])

BOOK ONE THE COMING OF THE MARTIANS I

 THE EVE OF THE WAR

 No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâs and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water



In [24]:
pd.read_csv('book_text.csv').iloc[20][0]

' And looking across space with instruments and intelligences such as we have scarcely dreamed of they see at its nearest distance only 35000000 of miles sunward of them a morning star of hope our own warmer planet green with vegetation and grey with water with a cloudy atmosphere eloquent of fertility with glimpses through its drifting cloud wisps of broad stretches of populous country and narrow navy-crowded seas'

In [25]:
data = TextLMDataBunch.from_csv('.', 'book_text.csv', text_cols=0)

In [26]:
data

TextLMDataBunch;

Train: LabelList (2434 items)
x: LMTextList
xxbos xxmaj the xxmaj shepperton guns spoke this time xxunk,xxbos xxmaj but i believed in him sufficiently to work with him all that morning until past midday at his digging,xxbos xxmaj he wanted the slit which permitted only one of us to xxunk through ; and so i had to xxunk watching them for a time while he xxunk that privilege,xxbos xxmaj on me and mine be the xxunk laid,xxbos âit would be curious to know how they live on another planet ; we might learn a thing or two
y: LMLabelList
,,,,
Path: .;

Valid: LabelList (609 items)
x: LMTextList
xxbos xxmaj perhaps they also prayed xxunk to xxmaj god,xxbos xxmaj even the birds were xxunk and as we hurried along i and the artilleryman talked in xxunk and looked now and again over our shoulders,xxbos a xxunk xxunk runs towards the foot of xxmaj maybury xxmaj hill and down this we xxunk,xxbos â xxmaj on that last xxunk their xxunk - writer xxunk very xxunk,xxbos xxmaj for a ti

In [27]:
data.train_ds[0]

(Text xxbos xxmaj the xxmaj shepperton guns spoke this time xxunk, EmptyLabel )

In [28]:
data.train_ds[1][0].data

array([  2,   5,  31,  14, 888,  15,  60, 889,  13, 305,  20,  60,  42,  17, 245, 181, 283, 692,  22,  37, 620])

In [29]:
data.valid_ds[1][0].data

array([   2,    5,  120,    9, 1021,   25,    0,   10,   21,   36,  469,  125,   14,   10,    9,  273,  653,   15,
          0,   10,  179,   87,   10,   84,   62,   86, 2235])

In [30]:
data.one_batch()[0].shape

torch.Size([64, 70])

In [31]:
x, y = data.one_batch()

In [32]:
x.shape, y.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [33]:
x[0], y[0]

(tensor([  24, 2063,   18,   13,   12, 1531, 1532,   11,  373,  128,   10,  891,
         2064,   38,   11,    9,  256, 2065, 1242, 2066,  556,   38,   11,   12,
          467,    2,    5,  183,    9,  168,   16,    0,   10,    9, 2067,  321,
         1044,  233,  780,  246,    2,    0,  557,  129,   88, 1045,    2,    5,
            9,  137,  198,    9,  247,   27,  399,   16, 2068,   20,   58,  219,
           89,  170,   15,   12,  234,   27, 1533,  289,  267,   22]),
 tensor([2063,   18,   13,   12, 1531, 1532,   11,  373,  128,   10,  891, 2064,
           38,   11,    9,  256, 2065, 1242, 2066,  556,   38,   11,   12,  467,
            2,    5,  183,    9,  168,   16,    0,   10,    9, 2067,  321, 1044,
          233,  780,  246,    2,    0,  557,  129,   88, 1045,    2,    5,    9,
          137,  198,    9,  247,   27,  399,   16, 2068,   20,   58,  219,   89,
          170,   15,   12,  234,   27, 1533,  289,  267,   22,   75]))

In [34]:
len(data.vocab.itos)

3088

In [35]:
nv = len(data.vocab.itos)

In [36]:
nh = 64

In [37]:
bs = 64

In [38]:
nv, nh, bs

(3088, 64, 64)

In [105]:
class BasicLanguageModel(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh) 
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        self.reset()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i]) 
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res_stacked = torch.stack(res, dim=1)
        out = self.h_o(res_stacked)
        return (out, res, res)
    
    def reset(self):
        self.h = torch.zeros(nh, nh).cuda()

In [97]:
mdl = BasicLanguageModel()

In [98]:
mdl

BasicLanguageModel(
  (i_h): Embedding(3088, 64)
  (h_h): Linear(in_features=64, out_features=64, bias=True)
  (h_o): Linear(in_features=64, out_features=3088, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [99]:
learn = LanguageLearner(data, model=mdl, metrics=accuracy)
# learn = Learner(data, model=mdl, metrics=accuracy)
# learn = language_model_learner(data, arch=AWD_LSTM, metrics=accuracy)

In [101]:
from fastai.text import *
learn.fit(20, lr=3e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.285446,4.950697,0.152288,00:00
1,5.115359,4.808184,0.183929,00:00
2,4.967049,4.701544,0.192188,00:01
3,4.834863,4.640527,0.198828,00:01
4,4.718391,4.635948,0.203348,00:00
5,4.610754,4.615658,0.201339,00:00
6,4.515334,4.611588,0.208147,00:00
7,4.424106,4.664527,0.206696,00:00
8,4.330434,4.673954,0.205915,00:00
9,4.242741,4.713938,0.198828,00:00


In [104]:
learn.predict("In the darkness ", n_words=20)

'In the darkness  and then it would be the that that are the three bulk overtaken him and stood about me a full'